<a href="https://colab.research.google.com/github/HrutvicBiramane/Learning-Github/blob/main/LSTM_CNN_%26_RECOMMENDATION.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install implicit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.9/8.9 MB 30.3 MB/s eta 0:00:00


In [ ]:
!pip install keras==2.4.3

  Attempting uninstall: keras
    Found existing installation: keras 3.3.3
    Uninstalling keras-3.3.3:
      Successfully uninstalled keras-3.3.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.16.1 requires keras>=3.0.0, but you have keras 2.4.3 which is incompatible.
tf-keras 2.15.1 requires tensorflow<2.16,>=2.15, but you have tensorflow 2.16.1 which is incompatible.


In [ ]:
!pip install tensorflow

In [ ]:
# Check TensorFlow version
import tensorflow as tf
print(tf.__version__)

# Check Keras version
import keras
print(keras.__version__)

2.16.1
3.3.3


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!wget http://nlp.stanford.edu/data/glove.6B.zip
!unzip glove.6B.zip -d "/content/drive/MyDrive/"

--2024-05-04 06:20:48--  http://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.6B.zip [following]
--2024-05-04 06:20:48--  https://nlp.stanford.edu/data/glove.6B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [following]
--2024-05-04 06:20:48--  https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip’

glov

In [ ]:
import pandas as pd
import numpy as np
import nltk
from nltk.corpus import stopwords
import string
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, Conv1D, LSTM, GlobalMaxPooling1D, Dense, Dropout, Bidirectional
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical
from scipy.sparse import csr_matrix
import implicit
import os
from nltk.sentiment import SentimentIntensityAnalyzer
from nltk.tokenize import sent_tokenize

# Setting OpenBLAS to use a single thread for performance stability
os.environ['OPENBLAS_NUM_THREADS'] = '1'
from threadpoolctl import threadpool_limits
threadpool_limits(limits=1, user_api='blas')

nltk.download('stopwords')
nltk.download('punkt')

# Load GloVe pre-trained word embeddings
def load_glove_embeddings(path):
    embeddings_index = {}
    with open(path, 'r', encoding='utf-8') as f:
        for line in f:
            values = line.split()
            word = values[0]
            coefs = np.asarray(values[1:], dtype='float32')
            embeddings_index[word] = coefs
    return embeddings_index

glove_embeddings = load_glove_embeddings('/content/drive/MyDrive/glove.6B.100d.txt')

# Load and preprocess data
print("Loading data...")
data = pd.read_csv('/content/drive/MyDrive/sentiment-analysis/Reviews-updated - Reviews.csv')
data.dropna(subset=['Reviews', 'Rating', 'Product Name'], inplace=True)

data['Sentiment'] = data['Rating'].apply(lambda x: 'positive' if x > 3 else 'negative' if x < 3 else 'neutral')

def preprocess_text(text):
    stop_words = set(stopwords.words('english'))
    text = text.translate(str.maketrans('', '', string.punctuation))
    tokens = [word for word in nltk.word_tokenize(text.lower()) if word.isalpha() and not word in stop_words]
    return ' '.join(tokens)

data['Processed_Reviews'] = data['Reviews'].apply(preprocess_text)

# Define aspect keywords
aspect_keywords = {
    'battery': ['battery', 'power', 'charge', 'charger'],
    'camera': ['camera', 'lens', 'megapixels', 'photo', 'picture'],
    'price': ['price', 'cost', 'expensive', 'cheap', 'value', 'worth'],
    'display': ['display', 'screen', 'resolution', 'bright', 'touchscreen', 'hd', '4k', 'oled'],
    'performance': ['performance', 'fast', 'slow', 'lag', 'memory', 'processor', 'speed', 'storage']
}

# Map aspects based on keywords in reviews
def map_aspects(review, aspect_keywords):
    present_aspects = {aspect: 0 for aspect in aspect_keywords.keys()}
    for aspect, keywords in aspect_keywords.items():
        if any(keyword in review.lower() for keyword in keywords):
            present_aspects[aspect] = 1
    return present_aspects

aspect_data = data['Processed_Reviews'].apply(lambda review: map_aspects(review, aspect_keywords))
aspect_df = pd.DataFrame(list(aspect_data))

# Encode sentiments and aspects
# label_encoder = LabelEncoder()
# data['Sentiment_Label'] = label_encoder.fit_transform(data['Sentiment'])
# Now encode 'Sentiment'
le = LabelEncoder()
data['Sentiment_Label'] = le.fit_transform(data['Sentiment'])

# Convert sentiment labels to one-hot encoded format
sentiment_labels = to_categorical(data['Sentiment_Label'])

# Prepare tokenizer and sequences
tokenizer = Tokenizer(num_words=5000)
tokenizer.fit_on_texts(data['Processed_Reviews'])
sequences = tokenizer.texts_to_sequences(data['Processed_Reviews'])
data_pad = pad_sequences(sequences, maxlen=100)

# Split data
# Prepare your data for splitting
X_train, X_test, y_train, y_test = train_test_split(data_pad, sentiment_labels, test_size=0.2, random_state=42)
aspect_train, aspect_test = train_test_split(aspect_df, test_size=0.2, random_state=42)

# Embedding matrix for GloVe
embedding_dim = 100
embedding_matrix = np.zeros((5000, embedding_dim))
for word, i in tokenizer.word_index.items():
    if i < 5000:
        embedding_vector = glove_embeddings.get(word)
        if embedding_vector is not None:
            embedding_matrix[i] = embedding_vector

# Model definition using GloVe Embeddings
input_text = Input(shape=(100,))
embedding_layer = Embedding(5000, 100, weights=[embedding_matrix], trainable=False)(input_text)
conv_layer = Conv1D(64, 3, activation='relu')(embedding_layer)
lstm_layer = Bidirectional(LSTM(64, return_sequences=True))(conv_layer)
pooling_layer = GlobalMaxPooling1D()(lstm_layer)
dropout_layer = Dropout(0.5)(pooling_layer)
outputs = [Dense(3, activation='softmax', name=f"{aspect}_sentiment")(dropout_layer) for aspect in aspect_keywords]
model = Model(inputs=input_text, outputs=outputs)
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

# Callbacks for training
early_stopping = EarlyStopping(monitor='val_loss', patience=5, verbose=1, restore_best_weights=True)
model_checkpoint = ModelCheckpoint('best_model.h5', monitor='val_accuracy', save_best_only=True, verbose=1)

# Convert aspect train/test data to one-hot
y_train_aspects = [to_categorical(aspect_train[aspect], num_classes=3) for aspect in aspect_keywords]
y_test_aspects = [to_categorical(aspect_test[aspect], num_classes=3) for aspect in aspect_keywords]

# Model training
print("Starting model training...")
model.fit(X_train, y_train_aspects, epochs=50, batch_size=32, validation_split=0.2, callbacks=[early_stopping, model_checkpoint])
print("Model training complete.")

# Model evaluation
print("Evaluating model...")
evaluation_results = model.evaluate(X_test, y_test_aspects)
print("Evaluation Results:", evaluation_results)

# ALS Model for recommendation
brand_encoder = LabelEncoder()
data['Brand_ID'] = brand_encoder.fit_transform(data['Product Name'])
num_brands = data['Brand_ID'].nunique()  # Number of unique brands
#num_sentiments = data['Sentiment_Label'].max() + 1  'Sentiment_Label' is zero-indexed
num_sentiments = data['Sentiment_Label'].max() + 1  # 'Sentiment_Label' is zero-indexed


aspect_rating_matrix = csr_matrix(
    (data['Rating'], (data['Brand_ID'], data['Sentiment_Label'])),
    shape=(num_brands, num_sentiments)  # Adjusted dimensions
)
# Print the matrix type to confirm
print("Matrix type:", type(aspect_rating_matrix))
print("Matrix shape:", aspect_rating_matrix.shape)

# Fit the ALS model
als_model = implicit.als.AlternatingLeastSquares(factors=50, regularization=0.1, iterations=20)
als_model.fit(aspect_rating_matrix.T)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


Loading data...
Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 100)]                0         []                            
                                                                                                  
 embedding (Embedding)       (None, 100, 100)             500000    ['input_1[0][0]']             
                                                                                                  
 conv1d (Conv1D)             (None, 98, 64)               19264     ['embedding[0][0]']           
                                                                                                  
 bidirectional (Bidirection  (None, 98, 128)              66048     ['conv1d[0][0]']              
 al)                                                                          

42/42 [==============================] - 15s 214ms/step - loss: 3.4020 - battery_sentiment_loss: 0.6462 - camera_sentiment_loss: 0.7123 - price_sentiment_loss: 0.6503 - display_sentiment_loss: 0.6949 - performance_sentiment_loss: 0.6983 - battery_sentiment_accuracy: 0.7502 - camera_sentiment_accuracy: 0.6875 - price_sentiment_accuracy: 0.7691 - display_sentiment_accuracy: 0.7079 - performance_sentiment_accuracy: 0.6611 - val_loss: 1.9784 - val_battery_sentiment_loss: 0.3725 - val_camera_sentiment_loss: 0.4510 - val_price_sentiment_loss: 0.3124 - val_display_sentiment_loss: 0.3882 - val_performance_sentiment_loss: 0.4543 - val_battery_sentiment_accuracy: 0.8343 - val_camera_sentiment_accuracy: 0.8675 - val_price_sentiment_accuracy: 0.8554 - val_display_sentiment_accuracy: 0.8916 - val_performance_sentiment_accuracy: 0.8193
Epoch 2/50
42/42 [==============================] - ETA: 0s - loss: 1.8907 - battery_sentiment_loss: 0.3399 - camera_sentiment_loss: 0.3796 - price_sentiment_loss: 0.

42/42 [==============================] - 5s 116ms/step - loss: 1.8907 - battery_sentiment_loss: 0.3399 - camera_sentiment_loss: 0.3796 - price_sentiment_loss: 0.3435 - display_sentiment_loss: 0.3905 - performance_sentiment_loss: 0.4372 - battery_sentiment_accuracy: 0.8777 - camera_sentiment_accuracy: 0.8642 - price_sentiment_accuracy: 0.8702 - display_sentiment_accuracy: 0.8430 - performance_sentiment_accuracy: 0.8113 - val_loss: 1.1171 - val_battery_sentiment_loss: 0.1833 - val_camera_sentiment_loss: 0.2040 - val_price_sentiment_loss: 0.1774 - val_display_sentiment_loss: 0.2457 - val_performance_sentiment_loss: 0.3067 - val_battery_sentiment_accuracy: 0.9488 - val_camera_sentiment_accuracy: 0.9669 - val_price_sentiment_accuracy: 0.9428 - val_display_sentiment_accuracy: 0.9277 - val_performance_sentiment_accuracy: 0.8735
Epoch 3/50
42/42 [==============================] - ETA: 0s - loss: 1.1312 - battery_sentiment_loss: 0.1902 - camera_sentiment_loss: 0.1782 - price_sentiment_loss: 0.2

42/42 [==============================] - 6s 155ms/step - loss: 1.1312 - battery_sentiment_loss: 0.1902 - camera_sentiment_loss: 0.1782 - price_sentiment_loss: 0.2190 - display_sentiment_loss: 0.2520 - performance_sentiment_loss: 0.2917 - battery_sentiment_accuracy: 0.9389 - camera_sentiment_accuracy: 0.9517 - price_sentiment_accuracy: 0.9268 - display_sentiment_accuracy: 0.9185 - performance_sentiment_accuracy: 0.8823 - val_loss: 0.8511 - val_battery_sentiment_loss: 0.1284 - val_camera_sentiment_loss: 0.1543 - val_price_sentiment_loss: 0.1241 - val_display_sentiment_loss: 0.1807 - val_performance_sentiment_loss: 0.2637 - val_battery_sentiment_accuracy: 0.9819 - val_camera_sentiment_accuracy: 0.9488 - val_price_sentiment_accuracy: 0.9518 - val_display_sentiment_accuracy: 0.9548 - val_performance_sentiment_accuracy: 0.8795
Epoch 4/50
42/42 [==============================] - ETA: 0s - loss: 0.8316 - battery_sentiment_loss: 0.1264 - camera_sentiment_loss: 0.1072 - price_sentiment_loss: 0.1

...

42/42 [==============================] - 7s 160ms/step - loss: 0.0934 - battery_sentiment_loss: 0.0152 - camera_sentiment_loss: 0.0180 - price_sentiment_loss: 0.0190 - display_sentiment_loss: 0.0190 - performance_sentiment_loss: 0.0221 - battery_sentiment_accuracy: 0.9962 - camera_sentiment_accuracy: 0.9955 - price_sentiment_accuracy: 0.9947 - display_sentiment_accuracy: 0.9962 - performance_sentiment_accuracy: 0.9932 - val_loss: 0.2374 - val_battery_sentiment_loss: 0.0517 - val_camera_sentiment_loss: 0.0354 - val_price_sentiment_loss: 0.0176 - val_display_sentiment_loss: 0.0690 - val_performance_sentiment_loss: 0.0637 - val_battery_sentiment_accuracy: 0.9880 - val_camera_sentiment_accuracy: 0.9910 - val_price_sentiment_accuracy: 0.9970 - val_display_sentiment_accuracy: 0.9819 - val_performance_sentiment_accuracy: 0.9759
Epoch 27/50
42/42 [==============================] - ETA: 0s - loss: 0.0720 - battery_sentiment_loss: 0.0114 - camera_sentiment_loss: 0.0072 - price_sentiment_loss: 0.

42/42 [==============================] - 5s 118ms/step - loss: 0.0720 - battery_sentiment_loss: 0.0114 - camera_sentiment_loss: 0.0072 - price_sentiment_loss: 0.0209 - display_sentiment_loss: 0.0124 - performance_sentiment_loss: 0.0201 - battery_sentiment_accuracy: 0.9985 - camera_sentiment_accuracy: 1.0000 - price_sentiment_accuracy: 0.9925 - display_sentiment_accuracy: 0.9977 - performance_sentiment_accuracy: 0.9962 - val_loss: 0.2572 - val_battery_sentiment_loss: 0.0549 - val_camera_sentiment_loss: 0.0347 - val_price_sentiment_loss: 0.0182 - val_display_sentiment_loss: 0.0753 - val_performance_sentiment_loss: 0.0741 - val_battery_sentiment_accuracy: 0.9880 - val_camera_sentiment_accuracy: 0.9940 - val_price_sentiment_accuracy: 0.9970 - val_display_sentiment_accuracy: 0.9819 - val_performance_sentiment_accuracy: 0.9729
Epoch 28/50
42/42 [==============================] - ETA: 0s - loss: 0.0527 - battery_sentiment_loss: 0.0087 - camera_sentiment_loss: 0.0072 - price_sentiment_loss: 0.

42/42 [==============================] - 7s 167ms/step - loss: 0.0527 - battery_sentiment_loss: 0.0087 - camera_sentiment_loss: 0.0072 - price_sentiment_loss: 0.0105 - display_sentiment_loss: 0.0100 - performance_sentiment_loss: 0.0164 - battery_sentiment_accuracy: 0.9977 - camera_sentiment_accuracy: 0.9992 - price_sentiment_accuracy: 0.9992 - display_sentiment_accuracy: 0.9985 - performance_sentiment_accuracy: 0.9947 - val_loss: 0.2355 - val_battery_sentiment_loss: 0.0538 - val_camera_sentiment_loss: 0.0362 - val_price_sentiment_loss: 0.0193 - val_display_sentiment_loss: 0.0737 - val_performance_sentiment_loss: 0.0525 - val_battery_sentiment_accuracy: 0.9880 - val_camera_sentiment_accuracy: 0.9910 - val_price_sentiment_accuracy: 0.9970 - val_display_sentiment_accuracy: 0.9819 - val_performance_sentiment_accuracy: 0.9789
Epoch 28: early stopping
Model training complete.
Evaluating model...
13/13 [==============================] - 1s 52ms/step - loss: 0.1607 - battery_sentiment_loss: 0.

/usr/local/lib/python3.10/dist-packages/implicit/utils.py:164: ParameterWarning: Method expects CSR input, and was passed csc_matrix instead. Converting to CSR took 0.0025184154510498047 seconds
  warnings.warn(


  0%|          | 0/20 [00:00<?, ?it/s]

In [ ]:
print("als_model:", als_model)
print("brand_encoder:", brand_encoder)


als_model: <implicit.cpu.als.AlternatingLeastSquares object at 0x7a45887b04c0>
brand_encoder: LabelEncoder()


In [ ]:
def recommend_products(aspect, top_n=10, als_model=None, brand_encoder=None):
    sia = SentimentIntensityAnalyzer()

    if als_model is None or brand_encoder is None:
        return pd.DataFrame(), "ALS model or brand encoder not provided."

    # Function logic using als_model and brand_encoder
    # ...


In [ ]:
def preprocess_product_name(product_name):
    stop_words = set(stopwords.words('english'))
    product_name = product_name.translate(str.maketrans('', '', string.punctuation))
    tokens = [word for word in nltk.word_tokenize(product_name.lower()) if word.isalpha() and word not in stop_words]
    return ' '.join(tokens)

data['Processed_Product_Name'] = data['Product Name'].apply(preprocess_product_name)

In [ ]:
for index, row in data.iterrows():
    print(row['Product Name'])


In [ ]:

print(data['Brand_ID'])
print(data['Product Name'])
print(data['Processed_Product_Name'])
print([row['Product Name']])

print(brand_encoder.classes_)


0        4
1        4
2        4
3        4
4        4
        ..
2067    19
2068    19
2069    19
2070    19
2071    19
Name: Brand_ID, Length: 2072, dtype: int64
0                 IPHONE 6S
1                 IPHONE 6S
2                 IPHONE 6S
3                 IPHONE 6S
4                 IPHONE 6S
               ...         
2067    Samsung Galaxy Note
2068    Samsung Galaxy Note
2069    Samsung Galaxy Note
2070    Samsung Galaxy Note
2071    Samsung Galaxy Note
Name: Product Name, Length: 2072, dtype: object
0                    iphone
1                    iphone
2                    iphone
3                    iphone
4                    iphone
               ...         
2067    samsung galaxy note
2068    samsung galaxy note
2069    samsung galaxy note
2070    samsung galaxy note
2071    samsung galaxy note
Name: Processed_Product_Name, Length: 2072, dtype: object
['Unknown Brand']
['BlackBerry 8800' 'BlackBerry Bold 9650' 'HP iPAQ 900' 'HTC Dream'
 'IPHONE 6S' 'JETHRO SC11' '

In [ ]:
import pandas as pd
from nltk.sentiment import SentimentIntensityAnalyzer
from nltk.tokenize import sent_tokenize
nltk.download('vader_lexicon')

def analyze_input_sentence():
    sia = SentimentIntensityAnalyzer()
    user_sentence = input("Enter a sentence for aspect-based sentiment analysis: ")

    detected_aspects = []
    for aspect, keywords in aspect_keywords.items():
        for keyword in keywords:
            if keyword in user_sentence.lower():
                detected_aspects.append((aspect, keyword))

    results = []
    if detected_aspects:
        for aspect, keyword in detected_aspects:
            # Extract the context around the keyword for sentiment analysis
            keyword_index = user_sentence.lower().index(keyword)
            start = max(0, keyword_index - 30)  # Adjust context window size as needed
            end = min(len(user_sentence), keyword_index + len(keyword) + 30)
            aspect_context = user_sentence[start:end]

            # Analyze sentiment for the aspect context
            sentiment_result = sia.polarity_scores(aspect_context)
            sentiment_score = sentiment_result['compound']
            sentiment = 'positive' if sentiment_score > 0.05 else 'negative' if sentiment_score < -0.05 else 'neutral'
            confidence = round((sentiment_score + 1) / 2 * 100, 2)  # Normalize and convert to percentage

            results.append({
                'Aspect': aspect,
                'Keyword': keyword,
                'Context': aspect_context,
                'Sentiment': sentiment,
                'Confidence': f"{confidence}%"
            })
    else:
        print("No known aspects detected in the sentence.")

    return pd.DataFrame(results)


# def analyze_input_sentence():
#     sia = SentimentIntensityAnalyzer()
#     user_sentence = input("Enter a sentence for aspect-based sentiment analysis: ")

#     detected_aspects = []
#     for aspect, keywords in aspect_keywords.items():
#         if any(keyword in user_sentence.lower() for keyword in keywords):
#             detected_aspects.append(aspect)

#     results = []
#     if detected_aspects:
#         # Analyze sentiment for the entire input sentence
#         sentiment_result = sia.polarity_scores(user_sentence)
#         sentiment_score = sentiment_result['compound']
#         sentiment = 'positive' if sentiment_score > 0.05 else 'negative' if sentiment_score < -0.05 else 'neutral'
#         confidence = round((sentiment_score + 1) / 2 * 100, 2)  # Normalize and convert to percentage

#         for aspect in detected_aspects:
#             results.append({
#                 'Aspect': aspect,
#                 'Sentence': user_sentence,
#                 'Sentiment': sentiment,
#                 'Confidence': f"{confidence}%"
#             })
#     else:
#         print("No known aspects detected in the sentence.")

#     return pd.DataFrame(results)

def recommend_products(aspect, top_n=10, als_model=als_model, brand_encoder=brand_encoder):
    sia = SentimentIntensityAnalyzer()

    # Filter reviews based on aspect keywords
    aspect_keywords_filter = '|'.join(aspect_keywords[aspect])
    filtered_reviews = data[data['Processed_Reviews'].str.contains(aspect_keywords_filter, case=False, regex=True)]

    results = []
    seen_review_segments = set()
    seen_brands = set()

    for index, row in filtered_reviews.iterrows():
        sentences = sent_tokenize(row['Reviews'])
        for sentence in sentences:
            if any(keyword in sentence.lower() for keyword in aspect_keywords[aspect]):
                sentiment_result = sia.polarity_scores(sentence)
                sentiment_score = sentiment_result['compound']
                sentiment = 'positive' if sentiment_score > 0.05 else 'negative' if sentiment_score < -0.05 else 'neutral'
                confidence = round((sentiment_score + 1) / 2 * 100, 2)  # Normalize and convert to percentage

                # Ensure the product name is valid
                product_name = row['Product Name']
                if pd.isnull(product_name):
                    continue  # Skip if product name is null or NaN

                # Use ALS model to get brand preferences
                if als_model and brand_encoder:
                    try:
                        # Transform product name to brand ID
                        brand_id = brand_encoder.transform([product_name])[0]

                        # Get similar items (brands) based on ALS model
                        recommended_brands = als_model.similar_items(brand_id, N=top_n)

                        # Extract recommended brand IDs and scores
                        recommended_brand_ids = [brand[0] for brand in recommended_brands]
                        recommended_scores = [brand[1] for brand in recommended_brands]

                        # Convert brand IDs back to product names and avoid duplicates
                        for brand_id, score in zip(recommended_brand_ids, recommended_scores):
                            # Round the brand ID to handle precision issues
                            rounded_brand_id = round(brand_id)
                            try:
                                recommended_product_name = brand_encoder.inverse_transform([rounded_brand_id])[0]
                                if recommended_product_name not in seen_brands:
                                    review_segment_key = (sentence, recommended_product_name)  # Unique key for each recommendation
                                    if review_segment_key not in seen_review_segments:
                                        results.append({
                                            'Product Name': recommended_product_name,
                                            'Review Segment': sentence,
                                            'Sentiment': sentiment,
                                            'Confidence': f"{confidence}%",
                                            'ALS Similarity Score': score
                                        })
                                        seen_review_segments.add(review_segment_key)
                                        seen_brands.add(recommended_product_name)
                            except Exception as e:
                                print(f"Error in transforming brand ID to product name: {e}")
                    except KeyError as e:
                        # Handle case where product name is not found in brand_encoder
                        print(f"KeyError: {e}")
                        results.append({
                            'Product Name': "Unknown Brand",
                            'Review Segment': sentence,
                            'Sentiment': sentiment,
                            'Confidence': f"{confidence}%"
                        })
                    except Exception as e:
                        # Handle any other exceptions
                        print(f"Exception: {e}")
                        results.append({
                            'Product Name': "Unknown Brand",
                            'Review Segment': sentence,
                            'Sentiment': sentiment,
                            'Confidence': f"{confidence}%"
                        })
                else:
                    results.append({
                        'Product Name': product_name,
                        'Review Segment': sentence,
                        'Sentiment': sentiment,
                        'Confidence': f"{confidence}%"
                    })

    results_df = pd.DataFrame(results)

    if not results_df.empty:
        top_products = results_df['Product Name'].value_counts().head(top_n).index.tolist()
        detailed_results = results_df[results_df['Product Name'].isin(top_products)].head(top_n)
        return detailed_results, None
    else:
        return pd.DataFrame(), "No relevant recommendations found."


#probably the correct one with unknown brand issue
# def recommend_products(aspect, top_n=10, als_model=als_model, brand_encoder=brand_encoder):
#     sia = SentimentIntensityAnalyzer()

#     # Filter reviews based on aspect keywords
#     aspect_keywords_filter = '|'.join(aspect_keywords[aspect])
#     filtered_reviews = data[data['Processed_Reviews'].str.contains(aspect_keywords_filter, case=False, regex=True)]

#     results = []

#     # print("Aspect Keywords:", aspect_keywords[aspect])
#     # print(filtered_reviews.head())  # Check the first few rows of filtered reviews

#     for index, row in filtered_reviews.iterrows():
#         # print("product Name:",row['Product Name'])
#         sentences = sent_tokenize(row['Reviews'])
#         for sentence in sentences:
#             if any(keyword in sentence.lower() for keyword in aspect_keywords[aspect]):
#                 sentiment_result = sia.polarity_scores(sentence)
#                 sentiment_score = sentiment_result['compound']
#                 sentiment = 'positive' if sentiment_score > 0.05 else 'negative' if sentiment_score < -0.05 else 'neutral'
#                 confidence = round((sentiment_score + 1) / 2 * 100, 2)  # Normalize and convert to percentage

#                 # Ensure the product name is valid
#                 product_name = row['Product Name']
#                 if pd.isnull(product_name):
#                     continue  # Skip if product name is null or NaN

#                 # Use ALS model to get brand preferences
#                 if als_model and brand_encoder:
#                     try:
#                         # Transform product name to brand ID
#                         brand_id = brand_encoder.transform(data['Product Name'])[0]

#                         # Get similar items (brands) based on ALS model
#                         recommended_brands = als_model.similar_items(brand_id, N=top_n)

#                         # Extract recommended brand IDs and scores
#                         recommended_brand_ids = [brand[0] for brand in recommended_brands]
#                         recommended_scores = [brand[1] for brand in recommended_brands]

#                         # Convert brand IDs back to product names
#                         recommended_product_names = []
#                         for brand_id in recommended_brand_ids:
#                             if brand_id in brand_encoder.classes_:
#                                 recommended_product_names.append(brand_encoder.inverse_transform([brand_id])[0])
#                             else:
#                                 recommended_product_names.append("Unknown Brand")

#                         for recommended_product, score in zip(recommended_product_names, recommended_scores):
#                             results.append({
#                                 'Product Name': recommended_product,
#                                 'Review Segment': sentence,
#                                 'Sentiment': sentiment,
#                                 'Confidence': f"{confidence}%",
#                                 'ALS Similarity Score': score
#                             })
#                     except KeyError:
#                         # Handle case where product name is not found in brand_encoder
#                         results.append({
#                             'Product Name': "Unknown Brand",
#                             'Review Segment': sentence,
#                             'Sentiment': sentiment,
#                             'Confidence': f"{confidence}%"
#                         })
#                 else:
#                     results.append({
#                         'Product Name': product_name,
#                         'Review Segment': sentence,
#                         'Sentiment': sentiment,
#                         'Confidence': f"{confidence}%"
#                     })

#     results_df = pd.DataFrame(results)

#     if not results_df.empty:
#         top_products = results_df['Product Name'].value_counts().head(top_n).index.tolist()
#         detailed_results = results_df[results_df['Product Name'].isin(top_products)].head(top_n)
#         return detailed_results, None
#     else:
#         return pd.DataFrame(), "No relevant recommendations found."


# def recommend_products(aspect, top_n=10):
#     sia = SentimentIntensityAnalyzer()

#     # Filter reviews based on aspect keywords
#     aspect_keywords_filter = '|'.join(aspect_keywords[aspect])
#     filtered_reviews = data[data['Processed_Reviews'].str.contains(aspect_keywords_filter, case=False, regex=True)]

#     results = []

#     for index, row in filtered_reviews.iterrows():
#         sentences = sent_tokenize(row['Reviews'])
#         for sentence in sentences:
#             if any(keyword in sentence.lower() for keyword in aspect_keywords[aspect]):
#                 sentiment_result = sia.polarity_scores(sentence)
#                 sentiment_score = sentiment_result['compound']
#                 sentiment = 'positive' if sentiment_score > 0.05 else 'negative' if sentiment_score < -0.05 else 'neutral'
#                 confidence = round((sentiment_score + 1) / 2 * 100, 2)  # Normalize and convert to percentage

#                 results.append({
#                     'Product Name': row['Product Name'],
#                     'Review Segment': sentence,
#                     'Sentiment': sentiment,
#                     'Confidence': f"{confidence}%"
#                 })

#     results_df = pd.DataFrame(results)

#     if not results_df.empty:
#         top_products = results_df['Product Name'].value_counts().head(top_n).index.tolist()
#         detailed_results = results_df[results_df['Product Name'].isin(top_products)].head(top_n)
#         return detailed_results, None
#     else:
#         return pd.DataFrame(), "No relevant recommendations found."

aspect_sentiments = analyze_input_sentence()
if not aspect_sentiments.empty:
    print("Detected aspects and their sentiments:")
    print(aspect_sentiments)
print("                           ")

aspect_of_interest = input("enter the aspect : ").lower()  # Example aspect
top_n_recommendations = 10  # Number of recommendations to fetch

recommended_results, error_message = recommend_products(aspect_of_interest, top_n=top_n_recommendations)

if not error_message:
    # Assuming recommended_results is a DataFrame with columns like 'Product Name', 'Review Segment', etc.
    print(f"Top {len(recommended_results)} recommendations for aspect '{aspect_of_interest}':")
    for index, row in recommended_results.iterrows():
        print(f"Product: {row['Product Name']}, Review Segment: {row['Review Segment']}, Sentiment: {row['Sentiment']}, Confidence: {row['Confidence']}")
else:
    print("Error:", error_message)

# Initial user input analysis
# Analyzing input sentence to detect aspects and their sentiments
#
#
#
#
#here code to take input will come
#
#
#
#

# def recommend_products(aspect, top_n=10):
#     sia = SentimentIntensityAnalyzer()

#     # Filter reviews based on aspect keywords
#     aspect_keywords_filter = '|'.join(aspect_keywords[aspect])
#     filtered_reviews = data[data['Processed_Reviews'].str.contains(aspect_keywords_filter, case=False, regex=True)]

#     # Dictionary to store aggregated sentiment scores for each brand
#     brand_sentiment_scores = {}

#     # Iterate through filtered reviews and calculate sentiment scores for each brand
#     for index, row in filtered_reviews.iterrows():
#         brand = row['Product Name']
#         if brand not in brand_sentiment_scores:
#             brand_sentiment_scores[brand] = {'sentiment_score': 0, 'count': 0}
#         sentences = sent_tokenize(row['Reviews'])
#         for sentence in sentences:
#             if any(keyword in sentence.lower() for keyword in aspect_keywords[aspect]):
#                 sentiment_result = sia.polarity_scores(sentence)
#                 sentiment_score = sentiment_result['compound']
#                 brand_sentiment_scores[brand]['sentiment_score'] += sentiment_score
#                 brand_sentiment_scores[brand]['count'] += 1

#     # Calculate aggregated sentiment scores for each brand
#     for brand in brand_sentiment_scores:
#         if brand_sentiment_scores[brand]['count'] > 0:
#             brand_sentiment_scores[brand]['average_sentiment'] = brand_sentiment_scores[brand]['sentiment_score'] / brand_sentiment_scores[brand]['count']
#         else:
#             brand_sentiment_scores[brand]['average_sentiment'] = 0

#     # Sort brands based on average sentiment score
#     sorted_brands = sorted(brand_sentiment_scores.items(), key=lambda x: x[1]['average_sentiment'], reverse=True)

#     results = []

#     # Select top-ranked brands for recommendations
#     for brand, sentiment_info in sorted_brands[:top_n]:
#         # Filter reviews for the selected brand
#         brand_reviews = filtered_reviews[filtered_reviews['Product Name'] == brand]
#         # Select a review segment containing aspect-related keywords
#         review_segment = brand_reviews.iloc[0]['Reviews']
#         # Perform sentiment analysis on the review segment
#         sentiment_result = sia.polarity_scores(review_segment)
#         sentiment_score = sentiment_result['compound']
#         sentiment = 'positive' if sentiment_score > 0.05 else 'negative' if sentiment_score < -0.05 else 'neutral'
#         confidence = round((sentiment_score + 1) / 2 * 100, 2)  # Normalize and convert to percentage

#         # Append recommendation information to results
#         results.append({
#             'Product Name': brand,
#             'Review Segment': review_segment,
#             'Sentiment': sentiment,
#             'Confidence': f"{confidence}%"
#         })

#     results_df = pd.DataFrame(results)

#     if results_df.empty:
#         print("No relevant recommendations found.")
#     else:
#         print(f"Top {len(results_df)} recommendations for aspect '{aspect}':")
#         print(results_df.to_string(index=False))


[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


Enter a sentence for aspect-based sentiment analysis: the camera quality is good but performance is average
Detected aspects and their sentiments:
        Aspect      Keyword  \
0       camera       camera   
1  performance  performance   

                                             Context Sentiment Confidence  
0           the camera quality is good but performan  positive     61.91%  
1  he camera quality is good but performance is a...  positive     61.91%  
                           
enter the aspect : performance
Top 10 recommendations for aspect 'performance':
Product: IPHONE 6S, Review Segment: Works well, occasional slowness, but overall a good phone., Sentiment: positive, Confidence: 82.98%
Product: BlackBerry 8800, Review Segment: Spectacular performance and design, great value compared to other famous brands, no compromises., Sentiment: positive, Confidence: 82.43%
Product: Nokia 5200, Review Segment: Offers unbeatable value, combining premium features with outstanding p

In [ ]:
for index in data.index:
    print(index, data.loc[index, 'Product Name'], data.loc[index, 'Reviews'])


0 IPHONE 6S "best camera.Highly recommend!"
1 IPHONE 6S Great display, clean setup, perfect size. Iphone did a great job!
2 IPHONE 6S Very pleased. Good camera!
3 IPHONE 6S Works well, occasional slowness, but overall a good phone. Love it!
4 IPHONE 6S Good replacement phone, battery issue but still works."
5 IPHONE 6S Disappointed with the purchase. Phone doesn't charge and has a cracked side. Requesting a refund.
6 IPHONE 6S Charging port and battery issues. Disappointed with initial condition.
7 IPHONE 6S "Bad Battery issues, had to trash it. Won't buy again."
8 IPHONE 6S Old phone, slow with apps, prone to freezing. Performance is lacking, despite decent camera and video quality.
9 IPHONE 6S Good battery, responsive touch. display occasionally goes black, needs multiple presses to re-illuminate.
10 IPHONE 6S Replacing previous problematic phone. So far, so good!
11 IPHONE 6S Great price, arrived in two days with only a small blemish. Fully functional. Highly recommend!
12 IPHONE 6S